In [66]:
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()
import modern_robotics as mr
import numpy as np

### Functions & variables from former tasks

#### Functions

In [90]:
def skew(v):
    return Matrix([[0, -v[2], v[1]],
                   [v[2], 0, -v[0]],
                   [-v[1], v[0], 0]])

def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3, :3] = exp3(twist[:3], theta)
    T[:3, 3] = (eye(3) * theta + (1 - cos(theta)) * omega +
                (theta - sin(theta)) * omega * omega) * v
    return T

def exp3(omega, theta):
    omega = skew(omega)
    R = eye(3) + sin(theta) * omega + (1 - cos(theta)) * omega * omega
    return R

def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

def rotX(alfa_im1):
    Rx = eye(4)
    Rx[1,1] =    cos(alfa_im1)
    Rx[1,2] =   -sin(alfa_im1)
    Rx[2,1] =    sin(alfa_im1)
    Rx[2,2] =    cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = eye(4)
    Rz[0,0] =    cos(alfa_im1)
    Rz[0,1] =   -sin(alfa_im1)
    Rz[1,0] =    sin(alfa_im1)
    Rz[1,1] =    cos(alfa_im1)
    return Rz

def transX(a_im1):
    trA = eye(4)
    trA[0,3] =  a_im1
    return trA

def transZ(d_i):
    trA = eye(4)
    trA[2,3] =  d_i
    return trA

def Js_maker(Slist, theta_list):
    n_joints = Slist.shape[1]
    Js = zeros(6, n_joints)
    
    for i in range(n_joints-1, -1, -1):
        if i==0: # legger til denne for å få Js[0] = S_sb[0], står i boka
            Js[:,i] = Slist[:,i]
                    
        else:
            T = exp6(Slist[:,i-1], theta_list[i-1])

            for j in range( i-2, -1, -1):
                T = exp6(Slist[:,j], theta_list[j]) * T

        Js[:,i] = Ad(T) * Slist[:,i]

    Js.simplify()
    return Js

def Jb_maker(Blist, theta_list):
    n_joints = Blist.shape[1]
    Jb = zeros(6, n_joints)

    Jb[:,n_joints-1] = Blist[:,n_joints-1]

    for i in range(n_joints-2, -1, -1):
        T = exp6(Blist[:,i+1], -theta_list[i+1])

        for j in range( i+2, n_joints-2):
            T = exp6(Blist[:,j], -theta_list[j]) * T
    
        Jb[:,i] = Ad(T) * Blist[:,i]
    
    # Jb.simplify()
    return Jb





#### Variables

In [89]:
th1, th2, th3, th4, th5, th6 = dynamicsymbols('theta_1, theta_2, theta_3, theta_4, theta_5, theta_6')

config = Matrix([[0,0,0,th1],[pi/2,0,0,th2 + pi/2],[0,455,0,th3],[pi/2, 35, -420, th4],[-pi/2,0,0,th5],[pi/2, 0, 0,th6]])
config

Mi = Matrix([[eye(4)]*6])

for i in range(6):
    if i == 1:
        Mi[:,4*i:4*(i+1)] = rotX(config[i,0]) * transX(config[i,1]) * transZ(config[i,2]) * rotZ(-pi/2) # We compansate for the rotation of -pi/2 done when finding the D-H parameters 
    else:
        Mi[:,4*i:4*(i+1)] = rotX(config[i,0]) * transX(config[i,1]) * transZ(config[i,2])

M = Matrix([[0, 0, -1, 445], [0, -1, 0, 0], [-1, 0, 0, -490], [0, 0, 0, 1]])

Ai = Matrix([[0,-1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]]) # This is a given matrix due to revolute joints

# S_sp = zeros(6)
# for i in range(6):
#     dot_sum = eye(4)
#     for n in range(i,-1,-1):
#         dot_sum = Mi[:,4*n:4*(n+1)] * dot_sum
#     S_skew = dot_sum * Ai * Inverse(dot_sum)
#     S_sp[0,i] = S_skew[2,1] 
#     S_sp[1,i] = S_skew[0,2] 
#     S_sp[2,i] = S_skew[1,0] 
#     S_sp[3,i] = S_skew[0,3] 
#     S_sp[4,i] = S_skew[1,3] 
#     S_sp[5,i] = S_skew[2,3] 
S_sp = Matrix([[0, 0, 0, -1, 0, -1], [0, -1, -1, 0, -1, 0], [1, 0, 0, 0, 0, 0], [0, 0, -455, 0, -490, 0], [0, 0, 0, 490, 0, 490], [0, -25, -25, 0, -445, 0]])


S_bp = Matrix([[-1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1], [0, -420, -420, 0, 0, 0], [-445, 0, 0, 0, 0, 0], [0, -490, -35, 0, 0, 0]])
S_sp, S_bp
# config

⎛⎡0   0    0    -1    0    -1 ⎤  ⎡ -1    0     0    0  0  0⎤⎞
⎜⎢                            ⎥  ⎢                         ⎥⎟
⎜⎢0  -1    -1    0    -1    0 ⎥  ⎢ 0     1     1    0  1  0⎥⎟
⎜⎢                            ⎥  ⎢                         ⎥⎟
⎜⎢1   0    0     0    0     0 ⎥  ⎢ 0     0     0    1  0  1⎥⎟
⎜⎢                            ⎥, ⎢                         ⎥⎟
⎜⎢0   0   -455   0   -490   0 ⎥  ⎢ 0    -420  -420  0  0  0⎥⎟
⎜⎢                            ⎥  ⎢                         ⎥⎟
⎜⎢0   0    0    490   0    490⎥  ⎢-445   0     0    0  0  0⎥⎟
⎜⎢                            ⎥  ⎢                         ⎥⎟
⎝⎣0  -25  -25    0   -445   0 ⎦  ⎣ 0    -490  -35   0  0  0⎦⎠

In [69]:
# S_sp = np.array(S_sp)
# S_sp

## Le task

In [92]:
Jb = Jb_maker(S_bp, [th1,th2,th3,th4,th4,th5,th6])


In [93]:
Jb

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅(-(420⋅sin(θ₂) + 490⋅cos(θ₂) - 490)⋅sin(
⎢                                                                             
⎢-(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅((420⋅sin(θ₂) + 490⋅cos(θ₂) - 490)⋅sin(
⎢                                                                             
⎢                                                                             
⎣                                                                             

                                                   

In [71]:
# Js2 = mr.JacobianSpace(np.array(S_sp.T),np.array([th1,th2,th3,th4,th4,th5,th6]))
# kukaIK = mr.IKinSpace()

In [100]:
Jb_12 = zeros(3,3) # Arm
Jb_21 = zeros(3,3) # Wrist

Jb_12 = Jb[0:3,3:6]
Jb_21 = Jb[3:6,0:3]

Jb_12, Jb_21


⎛                        ⎡(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅(-(420⋅sin(θ₂) 
⎜⎡-sin(θ₄)  sin(θ₅)  0⎤  ⎢                                                    
⎜⎢                    ⎥  ⎢-(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅((420⋅sin(θ₂) 
⎜⎢   0      cos(θ₅)  0⎥, ⎢                                                    
⎜⎢                    ⎥  ⎢                                                    
⎝⎣cos(θ₄)      0     1⎦  ⎣                                                    

+ 490⋅cos(θ₂) - 490)⋅sin(θ₃) - (490⋅sin(θ₂) - 420⋅cos(θ₂) + 420)⋅cos(θ₃) - 35⋅
                                                                              
+ 490⋅cos(θ₂) - 490)⋅sin(θ₃) + (490⋅sin(θ₂) - 420⋅cos(θ₂) + 420)⋅cos(θ₃) + 35⋅
                                                                              
                                                                              
                                                                              

sin(θ₃) + 420⋅cos(θ₃) - 420)⋅sin(θ₄) + (sin(θ₂)⋅co

In [105]:
det_Jb_12 = Jb_12.det()
det_Jb_12

-sin(θ₄)⋅cos(θ₅)

In [103]:
det_Jb_21 = Jb_21.det()
det_Jb_21

              2        3        4                    2        3        2      
- 74088000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄) - 148176000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄)⋅c

  2                   2        3        2                   2        3        
os (θ₄) + 80776500⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄) - 74088000⋅sin (θ₂)⋅sin (θ₃)⋅cos

4                   2        3        2                  2        2        4  
 (θ₄) + 80776500⋅sin (θ₂)⋅sin (θ₃)⋅cos (θ₄) - 6174000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ

                        2        2        4                   2        2      
₄)⋅cos(θ₃) + 6174000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄) - 12348000⋅sin (θ₂)⋅sin (θ₃)⋅s

  2                2                   2        2        2                    
in (θ₄)⋅cos(θ₃)⋅cos (θ₄) + 86436000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄)⋅cos(θ₃) + 12348

       2        2        2        2                  2        2        2      
000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄)⋅cos (θ₄) - 6174000⋅sin (θ₂)⋅sin (θ₃)⋅sin (θ₄) -

            2        2                4       

In [99]:
Jb

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅(-(420⋅sin(θ₂) + 490⋅cos(θ₂) - 490)⋅sin(
⎢                                                                             
⎢-(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅((420⋅sin(θ₂) + 490⋅cos(θ₂) - 490)⋅sin(
⎢                                                                             
⎢                                                                             
⎣                                                                             

                                                   

In [107]:
solutionArm = solve(det_Jb_12,th1,th2,th3,th4,th4,th5,th6)
solutionArm

⎡                  ⎧    π⎫  ⎧    3⋅π⎫⎤
⎢{θ₄: 0}, {θ₄: π}, ⎨θ₅: ─⎬, ⎨θ₅: ───⎬⎥
⎣                  ⎩    2⎭  ⎩     2 ⎭⎦

In [98]:
solutionWrist = solve(det_Jb_21,th1,th2,th3,th4,th4,th5,th6) # denne funker ikke enda

KeyboardInterrupt: 

In [79]:
solutionArm, solutionWrist

[{θ₄: 0}]